In [1]:
import searoute
import pandas as pd
import random
import folium

In [2]:
ports_df = pd.read_csv('Data.csv')                                      # Load the ports data from CSV

ports_df.head(3)

,World Port Index Number,Region Name,Main Port Name,Alternate Port Name,UN/LOCODE,Country Code,World Water Body,Sailing Direction or Publication,Publication Link,Standard Nautical Chart,...,Supplies - Fuel Oil,Supplies - Diesel Oil,Supplies - Aviation Fuel,Supplies - Deck,Supplies - Engine,Repairs,Dry Dock,Railway,Latitude,Longitude
0,7950.0,United States E Coast -- 6585,Maurer,,,United States,North Atlantic Ocean,U.S. Coast Pilot 2 - Atlantic Coast: Cape Cod ...,https://nauticalcharts.noaa.gov/publications/c...,12331,...,Yes,Yes,Unknown,Yes,Yes,Moderate,Unknown,Unknown,40.533333,-74.250000
1,52235.0,Sulawesi -- 51970,Mangkasa Oil Terminal,,,Indonesia,Teluk Bone; Banda Sea; South Pacific Ocean,Sailing Directions Pub. 163 (Enroute) - Borneo...,https://msi.geo.nga.mil/api/publications/downl...,,...,No,No,Unknown,No,No,NaN,Unknown,Unknown,-2.733333,121.066667
2,47620.0,Madagascar -- 47350,Iharana,,,Madagascar,Indian Ocean,Sailing Directions Pub. 171 (Enroute) - East A...,https://msi.geo.nga.mil/api/publications/downl...,61560,...,No,No,Unknown,No,No,Emergency Only,Unknown,Unknown,-13.350000,50.000000


In [3]:
# src/route_generator.py
import pandas as pd
import random
from searoute import searoute

def load_ports(csv_path="Data.csv"):
    return pd.read_csv(csv_path)

def select_ports(ports_df):
    return ports_df.sample(2)

def generate_route(port1, port2):
    return searoute((port1['longitude'], port1['latitude']), (port2['longitude'], port2['latitude']))


In [4]:
# src/ais_simulator.py
import time
import uuid
from pyais import encode_msg
from datetime import datetime

def simulate_positions(waypoints, speed_knots=10, interval_minutes=5):
    # Generate time-stamped positions from waypoints
    positions = []
    mmsi = int(str(uuid.uuid4().int)[:9])  # MMSI is 9 digits

    for i, (lon, lat) in enumerate(waypoints):
        timestamp = datetime.utcnow().isoformat()
        positions.append({
            "mmsi": mmsi,
            "lat": lat,
            "lon": lon,
            "timestamp": timestamp,
        })
    return positions

def encode_ais(position):
    return encode_msg({
        'msg_type': 1,
        'mmsi': position['mmsi'],
        'lat': position['lat'],
        'lon': position['lon']
    }).as_nmea()


In [5]:
# src/websocket_server.py
import asyncio
import json
import websockets

async def playback_server(positions, speed=1.0):
    async def send(websocket, path):
        for pos in positions:
            message = {
                "message": "AIVDM",
                "mmsi": pos['mmsi'],
                "timestamp": pos['timestamp'],
                "payload": encode_ais(pos)
            }
            await websocket.send(json.dumps(message))
            if speed > 0:
                await asyncio.sleep(300 / speed)  # 5 min in seconds
    return await websockets.serve(send, "localhost", 8765)

# In main script
# asyncio.run(playback_server(simulated_positions, speed=2.0))


In [6]:
# src/db/models.py
from sqlalchemy import Column, String, Float, DateTime, create_engine
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

class AISMessage(Base):
    __tablename__ = 'ais_messages'
    mmsi = Column(String, primary_key=True)
    timestamp = Column(DateTime, primary_key=True)
    lat = Column(Float)
    lon = Column(Float)
    payload = Column(String)

# src/db/init_db.py
def init_db(uri="sqlite:///ais.db"):
    engine = create_engine(uri)
    Base.metadata.create_all(engine)


C:\Users\RINURAGAVI\AppData\Local\Temp\ipykernel_56728\2853767613.py:5: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [7]:
# src/websocket_client.py

import json
import websockets
from sqlalchemy.orm import sessionmaker
# from pyais.stream import AISStream
from pyais import decode

# decoded = decode(data['payload'])
# print(decoded)


async def receiver(uri="ws://localhost:8765"):
    async with websockets.connect(uri) as websocket:
        while True:
            data = json.loads(await websocket.recv())
            try:
                decoded = AISStream.from_string(data['payload']).decode()
                # Insert into DB
            except Exception as e:
                print("Error parsing message:", e)


In [8]:
# src/analytics.py

from geopy.distance import geodesic

def calculate_distance(points):
    total = 0
    for i in range(1, len(points)):
        total += geodesic(
            (points[i-1].lat, points[i-1].lon),
            (points[i].lat, points[i].lon)
        ).nautical
    return total


### Trial to plot the sea route

#### Chennai (India) to Bunbury (Australia)

In [9]:
import pandas as pd

ports_df = pd.read_csv("Data.csv")   

ports_df = ports_df.dropna(subset=["Country Code"])                                 # Drop rows with missing Country Code to avoid NaNs in the result

unique_country_codes = ports_df["Country Code"].unique()                            # Get unique country codes

print(unique_country_codes)


['United States' 'Indonesia' 'Madagascar' 'Tanzania' 'Vanuatu'
 'Papua New Guinea' 'Palau' 'Federated States of Micronesia' 'Turkey'
 'Canada' 'Gibraltar' 'Norway' 'Brazil' 'Saudi Arabia' 'Latvia' 'Denmark'
 'Ukraine' 'Saint Helena, Ascension, and Tristan da Cunha' 'Thailand'
 'Greece' 'United Kingdom' 'Spain' 'Italy' 'Iran' 'Norfolk Island'
 'Malaysia' 'Sudan' 'Philippines' 'Russia' 'Algeria' 'Senegal'
 'Equatorial Guinea' 'Chile' 'India' 'Antarctica' 'Greenland' 'Nigeria'
 'Gabon' 'New Zealand' 'Kiribati' 'Sweden' 'China' 'South Korea' 'Japan'
 'France' 'Sint Maarten' 'Egypt' 'French Polynesia' 'Taiwan' 'Burma'
 'United Arab Emirates' 'Dominican Republic' 'Cambodia' 'Wake Island'
 'Reunion' 'Poland' 'Honduras' 'Angola'
 'South Georgia and South Sandwich Islands' 'Guinea' 'Guinea-Bissau'
 'Ireland' 'Croatia' 'Faroe Islands' 'Belgium' 'Netherlands' 'Germany'
 'The Bahamas' 'Puerto Rico' 'Cuba' 'Israel' 'Somalia' 'Kenya' 'Romania'
 'Monaco' 'Libya' 'Portugal' 'Congo (Brazzaville)' 'Libe

In [10]:
import pandas as pd

ports_df = pd.read_csv("Data.csv")

australia_ports = ports_df[ports_df['Country Code'].str.lower() == 'australia']  # Filter ports where Country Code is australia (case-insensitive just in case)

australia_ports.head(3)


,World Port Index Number,Region Name,Main Port Name,Alternate Port Name,UN/LOCODE,Country Code,World Water Body,Sailing Direction or Publication,Publication Link,Standard Nautical Chart,...,Supplies - Fuel Oil,Supplies - Diesel Oil,Supplies - Aviation Fuel,Supplies - Deck,Supplies - Engine,Repairs,Dry Dock,Railway,Latitude,Longitude
329,54943.0,Tasmania -- 54700,Beauty Point,,AU BYP,Australia,Bass Strait; South Pacific Ocean,Sailing Directions Pub. 127 (Enroute) - East C...,https://msi.geo.nga.mil/api/publications/downl...,,...,Yes,Yes,Unknown,Unknown,Unknown,Limited,Unknown,Unknown,-41.150000,146.816667
447,54325.0,Australia -- 53290,Port Bonython,Stony Point,AU PBY,Australia,Spencer Gulf; Indian Ocean,Sailing Directions Pub. 175 (Enroute) - North ...,https://msi.geo.nga.mil/api/publications/downl...,,...,No,No,Unknown,Unknown,Unknown,Limited,Unknown,Unknown,-33.016667,137.766667
451,54290.0,Australia -- 53290,Port Pirie,,AU PPI,Australia,Spencer Gulf; Indian Ocean,Sailing Directions Pub. 175 (Enroute) - North ...,https://msi.geo.nga.mil/api/publications/downl...,,...,Unknown,Yes,Unknown,No,No,Limited,Unknown,Small,-33.183333,138.016667


In [11]:

ports_df = pd.read_csv("Data.csv")

india_ports = ports_df[ports_df['Country Code'].str.lower() == 'india']     # Filter ports where Country Code is India (case-insensitive just in case)

india_ports.head(3)


,World Port Index Number,Region Name,Main Port Name,Alternate Port Name,UN/LOCODE,Country Code,World Water Body,Sailing Direction or Publication,Publication Link,Standard Nautical Chart,...,Supplies - Fuel Oil,Supplies - Diesel Oil,Supplies - Aviation Fuel,Supplies - Deck,Supplies - Engine,Repairs,Dry Dock,Railway,Latitude,Longitude
92,49460.0,India East Coast -- 49310,Machilipatnam,,,India,Bay of Bengal; Indian Ocean,Sailing Directions Pub. 173 (Enroute) - India ...,https://msi.geo.nga.mil/api/publications/downl...,63280,...,Unknown,Unknown,Unknown,Unknown,Unknown,Limited,Unknown,Small,16.150000,81.166667
587,48620.0,India West Coast -- 48610,Mandvi,,IN MDV,India,Arabian Sea; Indian Ocean,Sailing Directions Pub. 173 (Enroute) - India ...,https://msi.geo.nga.mil/api/publications/downl...,63062,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,22.833333,69.350000
1744,48790.0,India West Coast -- 48610,Hazira,,IN HZA,India,Arabian Sea; Indian Ocean,Sailing Directions Pub. 173 (Enroute) - India ...,https://msi.geo.nga.mil/api/publications/downl...,63091,...,No,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,21.083333,72.633333


In [12]:

ports_df = pd.read_csv('Data.csv')                                      # Load the ports data from CSV

ports_df.head(3)


,World Port Index Number,Region Name,Main Port Name,Alternate Port Name,UN/LOCODE,Country Code,World Water Body,Sailing Direction or Publication,Publication Link,Standard Nautical Chart,...,Supplies - Fuel Oil,Supplies - Diesel Oil,Supplies - Aviation Fuel,Supplies - Deck,Supplies - Engine,Repairs,Dry Dock,Railway,Latitude,Longitude
0,7950.0,United States E Coast -- 6585,Maurer,,,United States,North Atlantic Ocean,U.S. Coast Pilot 2 - Atlantic Coast: Cape Cod ...,https://nauticalcharts.noaa.gov/publications/c...,12331,...,Yes,Yes,Unknown,Yes,Yes,Moderate,Unknown,Unknown,40.533333,-74.250000
1,52235.0,Sulawesi -- 51970,Mangkasa Oil Terminal,,,Indonesia,Teluk Bone; Banda Sea; South Pacific Ocean,Sailing Directions Pub. 163 (Enroute) - Borneo...,https://msi.geo.nga.mil/api/publications/downl...,,...,No,No,Unknown,No,No,NaN,Unknown,Unknown,-2.733333,121.066667
2,47620.0,Madagascar -- 47350,Iharana,,,Madagascar,Indian Ocean,Sailing Directions Pub. 171 (Enroute) - East A...,https://msi.geo.nga.mil/api/publications/downl...,61560,...,No,No,Unknown,No,No,Emergency Only,Unknown,Unknown,-13.350000,50.000000


In [13]:

ports_df = pd.read_csv('Data.csv')

ports_df['World Port Index Number'] = pd.to_numeric(ports_df['World Port Index Number'], errors='coerce') # Ensure numeric type for WPIN (in case it's read as string)

port_chennai = ports_df[ports_df['World Port Index Number'] == 49450.0].iloc[0]         # Filter chennai and bunbury using World Port Index Numbers
port_bunbury = ports_df[ports_df['World Port Index Number'] == 54500.0].iloc[0]

port_chennai_coords = (port_chennai['Latitude'], port_chennai['Longitude'])          # Extract coordinates
port_bunbury_coords = (port_bunbury['Latitude'], port_bunbury['Longitude'])

print(f"Selected Ports: {port_chennai['Main Port Name']} to {port_bunbury['Main Port Name']}")
print(f"Coordinates: {port_chennai_coords} to {port_bunbury_coords}")


Selected Ports: Chennai (Madras) to Bunbury
Coordinates: (13.1, 80.3) to (-33.316667, 115.633333)


In [14]:
import numpy as np

def interpolate_route(start_coords, end_coords, num_points=10):

    """
    Interpolates a route between two coordinates (start_coords and end_coords).
    Args:
    - start_coords: tuple of (lat, lon) for the start port.
    - end_coords: tuple of (lat, lon) for the end port.
    - num_points: The number of interpolated points.
    Returns:
    - A list of interpolated points (lat, lon).
    """
    
    latitudes = np.linspace(start_coords[0], end_coords[0], num_points)
    longitudes = np.linspace(start_coords[1], end_coords[1], num_points)
    
    route = list(zip(latitudes, longitudes))
    return route

vessel_route = interpolate_route(port_chennai_coords, port_bunbury_coords)          # Generate a simulated route using interpolation

print("Generated Vessel Route (Interpolated):")                                     # Print the interpolated route
for point in vessel_route:
    print(f"Latitude: {point[0]}, Longitude: {point[1]}")



Generated Vessel Route (Interpolated):
Latitude: 13.1, Longitude: 80.3
Latitude: 7.942592555555555, Longitude: 84.22592588888888
Latitude: 2.78518511111111, Longitude: 88.15185177777778
Latitude: -2.372222333333335, Longitude: 92.07777766666666
Latitude: -7.52962977777778, Longitude: 96.00370355555555
Latitude: -12.687037222222225, Longitude: 99.92962944444444
Latitude: -17.844444666666668, Longitude: 103.85555533333333
Latitude: -23.001852111111113, Longitude: 107.78148122222223
Latitude: -28.159259555555558, Longitude: 111.70740711111111
Latitude: -33.316667, Longitude: 115.633333


In [15]:

map_center = [port_chennai_coords[0], port_chennai_coords[1]]                            # Create a map centered around the first port
mymap = folium.Map(location=map_center, zoom_start=5)

folium.Marker(location=port_chennai_coords, popup=f"Port: {port_chennai['Main Port Name']}").add_to(mymap)        # Plot the ports on the map
folium.Marker(location=port_bunbury_coords, popup=f"Port: {port_bunbury['Main Port Name']}").add_to(mymap)

for point in vessel_route:
    folium.CircleMarker(location=point, radius=2, color="blue").add_to(mymap)            # Plot the interpolated route

mymap.save("interpolated_vessel_route.html")                                             # Save the map to an HTML file
